In [29]:
import numpy as np
import string

diccionario para guardar las palabras iniciales asignando tambien la concurrencia de la palabra y las palabras de primer orden y segundo orden, ejemplo:

frases:
la casa del perro
el vino esta frio
la viuda del general


initial_word = {"la":2, "el":1}
first_order = {"la":["casa", "viuda"],
                "casa":["del"], "del":["perro", "general"],
                "perro":["end"],}

second_order = {("la", "casa"): ["del"]}

In [30]:
initial_word = {}
first_order = {}
second_order = {}

In [31]:
def remove_punctuation(s):
    return s.translate(str.maketrans("","", string.punctuation))

In [32]:
#la funcion pide, un diccionario, key y valor
def add2dict(d, k, v):
    if k not in d:
        d[k] = []
    d[k].append(v)

In [33]:
with open("benedetti.txt", "r", encoding= "utf-8") as file:
    for line in file:
        #print(line)
        #transformar cada linea en token, eliminando puntuacion y volviendo todo en minuscula
        tokens = remove_punctuation(line.rstrip().lower()).split()
        #print(tokens)
    
        T = len(tokens)
        #print(f"Tamaño de la fila:  {T}")
        #recorer los elementos de las fila
        for i in range(T):
            token = tokens[i]
            if i == 0:
                initial_word[token] = initial_word.get(token, 0.) + 1
                #print(f"Palabra incial: {token}")
                
            else: 
                #para guardar la palabra anterior
                t_1 = tokens[i-1]
                if i == T - 1:
                    add2dict(second_order, (t_1, token), "END")
                if i == 1:
                    add2dict(first_order, t_1, token)
                else:
                    t_2 = tokens[i-2]
                    add2dict(second_order,(t_2, t_1), token)

In [34]:
#initial_word
#second_order
#first_order

Para calcular ahora las probabilidades, vamos a normalizar

In [35]:
initial_total = sum(initial_word.values())
print(initial_total)

for t, c in initial_word.items():
    initial_word[t] = c / initial_total

1456.0


In [36]:
#initial_word

In [37]:
def list2pdict(ts):
    d = {}
    n = len(ts)
    
    #contar la ocurrencia de cada elemento en la lista
    for t in ts:
        d[t] = d.get(t, 0.) + 1
    
    #convertir los conteos en probabilidades relativas 
    for t, c in d.items():
        d[t] = c / n
        
    #devolver el diccionario de probabilidaddes
    return d 

In [38]:
for t_1, ts in first_order.items():
    first_order[t_1] = list2pdict(ts)

In [39]:
#first_order

In [40]:
for k, ts in second_order.items():
    second_order[k] = list2pdict(ts)

In [41]:
#second_order

In [42]:
def example_word(d, imprimir):
    #genera un numero aleatorio en el rango 0 - 1
    p0 = np.random.random()
    if(imprimir == 1):
        print(f"p0: {p0}")
        
    #inicializa una variable para realizar la suma acumulativa de probabilidades
    cumulative = 0
    if (imprimir == 1):
        print(f"prob acumulada: {cumulative}")
        
    #ciclo para recorrer cada clave t y su probabilidad p en diccionario d
    for t, p in d.items():
        #agregar la probabilidad aactual al valor acumulativo
        cumulative += p
        if(imprimir == 1):
            print(f"item: {t}, Prob: {p}")
            print(f"prob acumulada: {cumulative}")
                
        #comprobar si e numero aleatorio es meno que la acumulacion de probabilidades
        if p0 < cumulative:
            return t

In [43]:
first_order["yo"]

{'no': 0.4,
 'estoy': 0.1,
 'misma': 0.1,
 'les': 0.1,
 'quiziera': 0.1,
 'las': 0.1,
 'despejaba': 0.1}

In [44]:
example_word(first_order["yo"], 1)

p0: 0.7580290392502802
prob acumulada: 0
item: no, Prob: 0.4
prob acumulada: 0.4
item: estoy, Prob: 0.1
prob acumulada: 0.5
item: misma, Prob: 0.1
prob acumulada: 0.6
item: les, Prob: 0.1
prob acumulada: 0.7
item: quiziera, Prob: 0.1
prob acumulada: 0.7999999999999999


'quiziera'

generador

In [ ]:
#size sera el tamaño de lineas que generamos el generador
def generator(size):
    for i in range(size):
        sentence = []
        
        #palabra inicial
        word0 = example_word(initial_word, 0)
        sentence.append(word0)
        
        #segunda palabra
        word1 = example_word(first_order[word0], 0)
        sentence.append(word1)
        
        #segundo orden hasta el fin
        while True:
            word2 = example_word(second_order[(word0, word1)], 0)
            if (word0, word1) not in second_order:
                print(f"Par no encontrado: ({word0}, {word1})")
            if word2 == "END":
                break
            sentence.append(word2)
            word0 = word1
            word1 = word2    
        print(" ".join(sentence))    

In [46]:
generator(10)

pensamos que no cae
y el puente de mi nuca
y el entusiasmo pobre y la ceniza ajena
la cosa es muchísimo más grave
sin embargo
y flamean como buscando el mar
te quiero
que sabe que no cae
hay oraciones hay pétalos hay ríos
y los solemnes adultos las celebran
